In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import csv
import os
import ast
from PIL import Image
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D
from keras.optimizers import SGD, Adam

In [4]:
sample = Image.open('/content/drive/MyDrive/BrainDead/dataset/challenge2/brain_dead_emotion_detection/Train/angry/Aishwarya_Rai67.png')
img_height, img_width = sample.size
print(img_height, img_width)

182 182


In [7]:
#Initialize Parameters
num_classes = 4
epochs = 20
batch_size = 32
num_features = 128
width, height = 182, 182

In [ ]:
# Build the model
model = Sequential()
#Module1 conv<<conv<<batchnorm<<relu<<maxpooling<<dropout
model.add(Conv2D(2*num_features,kernel_size=(3,3),padding='same',data_format='channels_last',input_shape=(width, height, 1)))
model.add(Conv2D(2*num_features,kernel_size=(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.2))
#Module2 conv<<conv<<batchnorm<<relu<<maxpool<<dropout
model.add(Conv2D(2*2*num_features,kernel_size=(3,3),padding='same'))
model.add(Conv2D(2*2*num_features,kernel_size=(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.2))
#Module3 conv<<conv<<batchnorm<<relu<<maxpool<<dropout
model.add(Conv2D(2*2*2*num_features,kernel_size=(1,1),padding='same'))
model.add(Conv2D(2*2*2*num_features,kernel_size=(1,1),strides=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.2))
#Module4 fc<<batchnorm<<fc<<batchnorm<<dropout<<softmax
model.add(Flatten())
model.add(Dense(units=128))
model.add(BatchNormalization())
model.add(Dense(units=128))
model.add(BatchNormalization())
model.add(Dropout(rate=0.2))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999),metrics=['accuracy'])
model.summary()

In [9]:
emotions = ['angry', 'happy', 'neutral', 'sad']
emotion_dict = {"angry": 0, "happy": 1, "neutral": 2, "sad": 3}

In [10]:
# Preparing the training dataset

train_dir_path = '/content/drive/MyDrive/BrainDead/dataset/challenge2/brain_dead_emotion_detection/Train'

train_data = []
for emotion in emotions:
  img_files = [f for f in os.listdir(f"{train_dir_path}/{emotion}") if f.endswith('.png')]

  for img_file in img_files:
    img = Image.open(os.path.join(f"{train_dir_path}/{emotion}", img_file))

    grayscale_image = img.convert("L")

    resized_image = grayscale_image.thumbnail((182, 182))

    pixels = list(grayscale_image.getdata())

    train_data.append((pixels, emotion))


In [11]:
# Preparing the testing dataset

test_dir_path = '/content/drive/MyDrive/BrainDead/dataset/challenge2/brain_dead_emotion_detection/Test'

test_data = []
for emotion in emotions:
  img_files = [f for f in os.listdir(f"{test_dir_path}/{emotion}") if f.endswith('.png')]

  for img_file in img_files:
    img = Image.open(os.path.join(f"{test_dir_path}/{emotion}", img_file))

    grayscale_image = img.convert("L")

    resized_image = grayscale_image.thumbnail((182, 182))

    pixels = list(grayscale_image.getdata())

    test_data.append((pixels, emotion))

In [12]:
df = pd.DataFrame(train_data, columns=['pixels', 'emotions'])
df = df.sample(frac = 1)

In [13]:
# Preprocessing the images and labels
X_train = []
y_train = []

for i in range(len(df)):
  pixels = df['pixels'][i]
  temp = []
  for pixel in pixels:
    temp.append(pixel)
  
  pixels = np.array(temp, dtype='float32')
  pixels /= 255. # normalize
  X_train.append(pixels)
  y_train.append(df['emotions'][i])


In [14]:
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], 182,182,1)

In [15]:
num_classes = len(np.unique(y_train))
for i in range(len(y_train)):
  y_train[i] = emotion_dict[y_train[i]]
y_train[0]

0

In [16]:
train_emotions = to_categorical(y_train, num_classes)

In [18]:
model.fit(X_train, train_emotions, validation_split=0.2, epochs=10, batch_size=32, verbose=True)

Epoch 1/10
46/46 [==============================] - 126s 2s/step - loss: 1.1869 - accuracy: 0.5104 - val_loss: 4.6256 - val_accuracy: 0.0000e+00
Epoch 2/10
46/46 [==============================] - 104s 2s/step - loss: 1.0377 - accuracy: 0.5912 - val_loss: 4.1356 - val_accuracy: 0.0000e+00
Epoch 3/10
46/46 [==============================] - 107s 2s/step - loss: 0.8953 - accuracy: 0.6547 - val_loss: 2.3557 - val_accuracy: 0.0000e+00
Epoch 4/10
46/46 [==============================] - 108s 2s/step - loss: 0.7880 - accuracy: 0.6851 - val_loss: 2.9639 - val_accuracy: 0.0000e+00
Epoch 5/10
46/46 [==============================] - 108s 2s/step - loss: 0.7148 - accuracy: 0.7120 - val_loss: 6.7235 - val_accuracy: 0.0000e+00
Epoch 6/10
46/46 [==============================] - 108s 2s/step - loss: 0.5877 - accuracy: 0.7548 - val_loss: 5.1627 - val_accuracy: 0.0000e+00
Epoch 7/10
46/46 [==============================] - 108s 2s/step - loss: 0.5321 - accuracy: 0.8004 - val_loss: 3.3355 - val_accura

In [19]:
df = pd.DataFrame(test_data, columns=['pixels', 'emotions'])
df = df.sample(frac = 1)

In [20]:
# Preprocessing the images and labels
X_test = []
y_test = []

for i in range(len(df)):
  pixels = df['pixels'][i]
  temp = []
  for pixel in pixels:
    temp.append(pixel)
  
  pixels = np.array(temp, dtype='float32')
  pixels /= 255. # normalize
  X_test.append(pixels)
  y_test.append(df['emotions'][i])

In [21]:
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], 182,182,1)

In [22]:
for i in range(len(y_test)):
  y_test[i] = emotion_dict[y_test[i]]

In [23]:
y_test[0]

0

In [24]:
test_emotions = to_categorical(y_test, num_classes)

In [25]:
model.metrics_names

['loss', 'accuracy']

In [ ]:
del(test_data)

In [ ]:
scores = model.evaluate(X_test, test_emotions, verbose=1)

12/12 [==============================] - 11s 914ms/step - loss: 1.9368 - accuracy: 0.4986


In [ ]:
scores

[2.5202136039733887, 0.4878048896789551]